In [ ]:
%pylab inline

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, auc

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

## Riaz cohort APM-cluster prediction

In [ ]:
dataset122 = pd.read_csv('cohort_liu.csv')

dataset40 = pd.read_csv('cohort_van_allen.csv')

frames = [dataset122,dataset40]

dataset_orig = pd.concat(frames)
dataset=dataset_orig[['cluster',"HLA_A","HLA_B","HLA_C","HLA_DRA", "HLA_DRB1","HLA_DQA1","HLA_DQB1","HLA_DPA1","HLA_DPB1",
                      'PSME1','TAPBP','NLRC5','PSMB10','TAP2','HLA_DRB6','HLA_DQA2','HLA_DQB2','CIITA','HLA_E','HLA_G','HLA_F','HLA_DMA','HLA_DOB']]
dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset.isna().sum()
dataset = dataset.dropna()
dataset.shape
dataset['cluster']=dataset['cluster'].astype('uint8')
dataset.shape
dataset.cluster.unique()

In [ ]:
# random sampling from each class since some classes may have low number of cases
frac=0.9
random_state=0

c1=dataset[dataset['cluster']==1]
c1_train_set = c1.sample(frac=frac, random_state=random_state)
c1_test_set = c1.drop(c1_train_set.index)

c2=dataset[dataset['cluster']==2]
c2_train_set = c2.sample(frac=frac, random_state=random_state)
c2_test_set = c2.drop(c2_train_set.index)

c3=dataset[dataset['cluster']==3]
c3_train_set = c3.sample(frac=frac, random_state=random_state)
c3_test_set = c3.drop(c3_train_set.index)

c4=dataset[dataset['cluster']==4]
c4_train_set = c4.sample(frac=frac, random_state=random_state)
c4_test_set = c4.drop(c4_train_set.index)

train_frames=[c1_train_set,c2_train_set,c3_train_set,c4_train_set]
train_set=pd.concat(train_frames)
test_frames=[c1_test_set,c2_test_set,c3_test_set,c4_test_set]
test_set=pd.concat(test_frames)

In [ ]:
train_features = train_set.copy()
test_features = test_set.copy()

train_labels = train_features.pop('cluster')
test_labels = test_features.pop('cluster')

X_train=pd.DataFrame.to_numpy(train_features)
y_train=pd.Series.to_numpy(train_labels)

X_test=pd.DataFrame.to_numpy(test_features)
y_test=pd.Series.to_numpy(test_labels)

X=np.concatenate((X_train,X_test),axis=0)
y=np.concatenate((y_train,y_test),axis=0)

In [ ]:
clf = LogisticRegression(solver="saga", random_state=0, max_iter=1500,C=8)
clf.fit(X_train, y_train)
y_score=clf.fit(X_train, y_train).predict_proba(X_test)
clf.fit(X_train, y_train).score(X_test, y_test)

In [ ]:
from sklearn import svm, datasets
from sklearn.metrics import auc
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import StratifiedKFold

# Run classifier with cross-validation and plot ROC curves
fprs = []
tprs = []
aucs = []

for i in range(1000):
    cv = StratifiedKFold(n_splits=10, shuffle=True)

    for i, (train, test) in enumerate(cv.split(X, y)):
        clf = LogisticRegression(solver="saga", random_state=0, max_iter=1500,C=8)

        y_score=clf.fit(X[train], y[train]).predict_proba(X[test])
        y_train_bin = label_binarize(y[train], classes=[1,2,3,4])
        y_test_bin = label_binarize(y[test], classes=[1,2,3,4])
        n_classes = y_train_bin.shape[1]
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])


        tprs.append(tpr)
        aucs.append(roc_auc)
        fprs.append(fpr)

In [ ]:
sum_microAUC=0
for i in range(len(aucs)):
    sum_microAUC += aucs[i]['micro']
print('mean micro AUC =',sum_microAUC/len(aucs))

In [ ]:
raw_dataset2 = pd.read_csv('cohort_riaz-pre-treatment.csv')


dataset_orig2 = raw_dataset2.copy()
dataset2=dataset_orig2[["HLA_A","HLA_B","HLA_C","HLA_DRA", "HLA_DRB1","HLA_DQA1","HLA_DQB1","HLA_DPA1","HLA_DPB1",
                      'PSME1','TAPBP','NLRC5','PSMB10','TAP2','HLA_DRB6','HLA_DQA2','HLA_DQB2','CIITA','HLA_E','HLA_G','HLA_F','HLA_DMA','HLA_DOB']]

dataset2.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset2.isna().sum()
dataset2 = dataset2.dropna()
X_pred=pd.DataFrame.to_numpy(dataset2)

In [ ]:
# Predict by logistic regression classifer
clf = LogisticRegression(solver="saga", random_state=0, max_iter=1500,C=8)
clf.fit(X, y)


y_pred=clf.predict(X_pred)
y_pred

In [ ]:
raw_dataset2['pred_cluster']=y_pred
raw_dataset2.to_csv('riaz_pred_cluster.csv', index = True)

# scRNA pseudobulk

In [ ]:
raw_dataset2 = pd.read_csv('scRNA-pseudobulk-for-clustering.csv')


dataset_orig2 = raw_dataset2.copy()
dataset2=dataset_orig2[["HLA_A","HLA_B","HLA_C","HLA_DRA", "HLA_DRB1","HLA_DQA1","HLA_DQB1","HLA_DPA1","HLA_DPB1",
                      'PSME1','TAPBP','NLRC5','PSMB10','TAP2','HLA_DRB6','HLA_DQA2','HLA_DQB2','CIITA','HLA_E','HLA_G','HLA_F','HLA_DMA','HLA_DOB']]

dataset2.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset2.isna().sum()
dataset2 = dataset2.dropna()

dataset2

In [ ]:
X_pred=pd.DataFrame.to_numpy(dataset2)

# Predict by logistic regression classifer
clf = LogisticRegression(solver="saga", random_state=0, max_iter=1500,C=8)
clf.fit(X, y)


y_pred=clf.predict(X_pred)
y_pred

In [ ]:
raw_dataset2['pred_cluster']=y_pred
raw_dataset2.to_csv('pseudobulk-clustering-pred.csv', index = True)